# Contents 
In this session we will cover the follwing:
1. NetworkX Overview
2. Adding Attributes to Nodes and Edges
3. The DiGraph Class - The Directed Graph
4. Community Detection 
5. Your Turn

## 1. NetworkX Overview

### Creating a Network

In [ ]:
import networkx as nx
from matplotlib import pyplot as plt
%matplotlib inline 

In [ ]:
G = nx.Graph()

G.add_node('A')
G.add_nodes_from(['B', 'C'])

G.add_edge('A', 'B')
G.add_edges_from([('B', 'C'), ('A', 'C')])

plt.figure(figsize=(7.5, 7.5))
nx.draw_networkx(G)

figure( ) is used to create a 7.5 by 7.5 inch figure to hold the visualization. 

In [ ]:
# Configure plotting in Jupyter
plt.rcParams.update({
    'figure.figsize': (7.5, 7.5),
    'axes.spines.right': False,
    'axes.spines.left': False,
    'axes.spines.top': False,
    'axes.spines.bottom': False})
# Seed random number generator
import random
from numpy import random as nprand
seed = hash("Network Science in Python") % 2**32
nprand.seed(seed)
random.seed(seed)

- Instead of configuring the figure size every time we want to display a figure we can edit the default by the command: plt.rcParams.update
- Visualization may look slightly different every time you run the code. This is because visualizations in NetworkX sometimes use randomized algorithms. The randomized algorithms can be configured to produce the same output each time by setting the random seed.  

### Add several nodes and edges at the same time

In [ ]:
G.add_edges_from([('B', 'D'), ('C', 'E')])
nx.draw_networkx(G)

### Undirected networks

- The code below stores the karate club network in G. 
- The visualization layout is then pre-calculated using spring_layout() and stored in karate_pos.  
- This will allow us to reuse the layout throughout the notebook. 
- Spring_layout() tries to place nodes closer together if they are connected by an edge
- There are many different layout methods, such as: spiral, shell and random, see the documentation for more details.
https://networkx.org/documentation/stable/reference/generated/networkx.drawing.layout.spring_layout.html
- Next, the call to draw_networkx() creates the following visualization:

In [ ]:
G = nx.karate_club_graph()
karate_pos = nx.spring_layout(G, k=0.3)
nx.draw_networkx(G, karate_pos)

In [ ]:
list(G.nodes)

In [ ]:
list(G.edges)

### Checking for nodes

In [ ]:
mr_hi = 0
mr_hi in G

In [ ]:
G.has_node(mr_hi)

In [ ]:
wild_goose = 1337
wild_goose in G

In [ ]:
G.has_node(wild_goose)

### Finding node neighbors

In [ ]:
list(G.neighbors(mr_hi))

In [ ]:
member_id = 1
(mr_hi, member_id) in G.edges

In [ ]:
G.has_edge(mr_hi, member_id)

In [ ]:
G.has_edge(33,0)

In [ ]:
john_a = 33
(mr_hi, john_a) in G.edges

In [ ]:
G.has_edge(mr_hi, john_a)

## Part 2:  Adding attributes to nodes and edges

In [ ]:
member_club = [
    0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
    0, 0, 0, 0, 1, 1, 0, 0, 1, 0,
    1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
    1, 1, 1, 1]

In [ ]:
for node_id in G.nodes:
    G.nodes[node_id]["club"] = member_club[node_id]

In [ ]:
G.add_node(11, club=0)

In [ ]:
G.nodes[mr_hi]

In [ ]:
G.nodes[john_a]

In [ ]:
node_color = [
    '#1f78b4' if G.nodes[v]["club"] == 0
    else '#33a02c' for v in G]

In [ ]:
nx.draw_networkx(G, karate_pos, label=True, node_color=node_color)

### Adding Edge Weights

In [ ]:
def tie_strength(G, v, w):
    # Get neighbors of nodes v and w in G
    v_neighbors = set(G.neighbors(v))
    w_neighbors = set(G.neighbors(w))
    # Return size of the set intersection
    return 1 + len(v_neighbors & w_neighbors)

In [ ]:
# Calculate weight for each edge
for v, w in G.edges: 
    G.edges[v, w]["weight"] = tie_strength(G, v, w)
# Store weights in a list
edge_weights = [G.edges[v, w]["weight"] for v, w in G.edges] 

In [ ]:
weighted_pos = nx.spring_layout(G, pos=karate_pos, k=0.3, weight="weight")

In [ ]:
# Draw network with edge color determined by weight
nx.draw_networkx(
    G, weighted_pos, width=8, node_color=node_color,
    edge_color=edge_weights, edge_vmin=0, edge_vmax=6, edge_cmap=plt.cm.Blues)


## Part 3: The DiGraph Class - The Directed Graph

In [ ]:
G = nx.read_gexf("data/knecht2008/klas12b-net-1.gexf", node_type=int)
student_pos = nx.spring_layout(G, k=1.5)
nx.draw_networkx(G, student_pos, arrowsize=20)

In [ ]:
list(G.neighbors(0))

In [ ]:
list(G.successors(0))

In [ ]:
list(G.predecessors(0))

In [ ]:
# Create undirected copies of G
G_either = G.to_undirected()
G_both = G.to_undirected(reciprocal=True)

# Set up a figure
plt.figure(figsize=(10,5))

# Draw G_either on left
plt.subplot(1, 2, 1)
nx.draw_networkx(G_either, student_pos)

# Draw G_both on right
plt.subplot(1, 2, 2)
nx.draw_networkx(G_both, student_pos)

## Part 4: Community Detection

### Givan Newman Algorithm

In [ ]:
from networkx.algorithms.community.centrality import girvan_newman

In [ ]:
G = nx.karate_club_graph()
communities = girvan_newman(G)

In [ ]:
node_groups = []
for com in next(communities):
  node_groups.append(list(com))

print(node_groups)

In [ ]:
color_map = []
for node in G:
    if node in node_groups[0]:
        color_map.append('blue')
    else: 
        color_map.append('green')  
nx.draw(G, node_color=color_map, with_labels=True)
plt.show()

### Louvain Algorithm 

In [ ]:
import community as community_louvain
import matplotlib.cm as cm

# compute the best partition
partition = community_louvain.best_partition(G)

# draw the graph
pos = nx.spring_layout(G)

# color the nodes according to their partition
cmap = cm.get_cmap('viridis', max(partition.values()) + 1)
nx.draw_networkx_nodes(G, pos, partition.keys(), node_size=40,
                       cmap=cmap, node_color=list(partition.values()))

nx.draw_networkx_edges(G, pos, alpha=0.5)
plt.show()

### Summary points
1. When creating a network from data, the most important questions to ask is what exactly the nodes and edges should represent.
2. There are many possibilities for the same dataset.
3. Some possibilities: 
    - Social relationships, such as friendships, romantic relationships, or even rivalries
    - Flows, such as information, people, money, fluids, or energy
    - Influence, such as scientific citations, software dependencies, or protein interactions
    - Connection, such as between networked computers, bones in a skeleton, neighboring countries, or railway lines
    - Interaction, such as predator-prey relationships or international treaties
    - Co-occurrence of words in text

***

## Part 5: Your Turn
Complete the code below as directed and submit it to the Blackboard page of the course.

Here is the content of a sample edge list network, representing a fictional subway system:
You can view the data file, example.edgelist in data folder
#Example edge list network

#source target 

Winegroom Uptown <br>
Winegroom Strawshop <br>
Uptown Strawshop <br>
Uptown Amazelake <br>
Strawshop Province <br>

Before reading the network file, we store the directory containing the data in data_dir:

In [ ]:
from pathlib import Path
data_dir = Path('.') / 'data'

Next, the read_edgelist( ) function creates a Graph class from the edge list file, like so:

In [ ]:
G = nx.read_edgelist(data_dir/'example.edgelist')

### Question 1: Visulaize this network as we did before, refer to the section [Undirected networks](#Undirected-networks)

In [ ]:
# Write your code here


### Question 2: Check whether the node "Province" is in the graph.

### Question 3: Check whether the node "Oakwood" is in the graph.

### Question 4: list all the nodes in the graph, and all the edges.

### Question 5: add the stations "Mall" and "MainStreet" in 1 step to the gaph and connect them to "uptown".

### Question 6: Find the neighbors of node "Uptown".

What we did so far, NetworkX assumed the edge list represents an undirected network and returned a Graph class accordingly. If a network is directed, the read_edgelist() function can also return a DiGraph class. The only necessary change is that the DiGraph class must be passed as the create_using parameter. For directed networks, the first node on each line of the edge list will be interpreted as the source and the second as the target. 

In [ ]:
# Read edge list

G = nx.read_edgelist(data_dir/'example.edgelist', create_using=nx.DiGraph)

### Question 7: Draw the directed graph we've just created.

### Question 8: Community Detection

Now we want to use the community detection algorithm explained above with a larger dataset. The following example uses a social network constructed from combining the online social networks of 10 individuals (McAuley & Leskovec, 2012). The following code loads and visualizes the network:

In [ ]:
# Load data file into network

G_social = nx.read_edgelist (data_dir / 'mcauley2012' / 'facebook_combined.txt')

# Calculate layout and draw

pos = nx.spring_layout(G_social, k=0.1)
nx.draw_networkx(
    G_social, pos=pos, node_size=0,
    edge_color="#333333", alpha=0.05, with_labels=False)

Your turn: Now applay community detection to G_social using Louvain algorithm as we did above.

In [ ]:
# compute the best partition
...